## Import

In [1]:
%pip install segmentation-models-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 637.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.7 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16427 sha256=7923a1fe6a3b8bfa2de7d64a29f13f011f2bb896ec813ab4b046bf090ab618f9
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=3c3aef49c426108ec0b6f2615cae2dc4deac17334f9de4888596889ac6c5f575
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c6

In [2]:
import os
import cv2
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from typing import List, Union
from joblib import Parallel, delayed

import segmentation_models_pytorch as smp
import argparse

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Utils

In [3]:
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

## Custom dataset

In [84]:
class SatelliteDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False, subset_indices=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer
        self.subset_indices = subset_indices

    def __len__(self):
        return len(self.subset_indices) if self.subset_indices else len(self.data)

    def __getitem__(self, idx):
        if self.subset_indices:
           idx = self.subset_indices[idx]

        img_path = self.data['img_path'][idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image

        mask_rle = self.data['mask_rle'][idx]
        mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

## Data Loader

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd /content/drive/MyDrive/ai project 4-1
#데이터있는 주소
#!unzip -qq "/content/drive/MyDrive/ai project 4-1/data.zip"

Mounted at /content/drive
/content/drive/MyDrive/ai project 4-1


In [86]:

transform = A.Compose(
    [
        A.CenterCrop(224, 224, p=0.1),

        #A.HorizontalFlip(p = 0.5),

        #A.IAAAdditiveGaussianNoise(p=0.2),
        #A.IAAPerspective(p=0.5),

        #A.OneOf([
            #A.CLAHE(p=1),
            #A.RandomBrightness(p = 1),
            #A.RandomGamma(p = 1)
        #], p = 0.5),

        #A.OneOf([
            #A.IAASharpen(p = 1),
            #A.Blur(blur_limit=3, p=1),
            #A.GaussianBlur(p = 1),
            #A.MotionBlur(blur_limit=3, p=1),
            #A.GaussNoise(p = 1)
        #], p = 0.5),

        #A.OneOf([
            #A.RandomContrast(p=1),
            #A.HueSaturationValue(p=1),
        #], p = 0.5),

        A.Resize(224, 224),

        A.Normalize(),
        ToTensorV2()
    ]
)

transform_test = A.Compose(
    [
        A.Resize(224, 224),

        A.Normalize(),
        ToTensorV2()
    ]
)

#dataset = SatelliteDataset(csv_file='train.csv', transform=transform)
#dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=2)

##train-validation 데이터 분할

train data를 train_rate 의 비율만큼 *분할함*.

단 transform이 train, val 둘다 적용됨

In [87]:
train_rate=0.9
train_val_dataset =  SatelliteDataset(csv_file='./train.csv', transform=None)

train_size = int(train_rate * len(train_val_dataset))
valid_size = len(train_val_dataset) - train_size

dataset, val_dataset = torch.utils.data.random_split(train_val_dataset, [train_size, valid_size])

data = SatelliteDataset(csv_file='./train.csv', transform=transform, subset_indices= dataset.indices)
val_data = SatelliteDataset(csv_file='./train.csv', transform=transform_test,subset_indices=val_dataset.indices)

dataloader = torch.utils.data.DataLoader(data, batch_size = 16, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size = 16, shuffle=False)

"""
dataset, val_dataset = torch.utils.data.random_split(train_val_dataset, [train_size, valid_size])
dataloader = torch.utils.data.DataLoader(dataset, batch_size = 16, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = 16, shuffle=False)
"""

'\ndataset, val_dataset = torch.utils.data.random_split(train_val_dataset, [train_size, valid_size])\ndataloader = torch.utils.data.DataLoader(dataset, batch_size = 16, shuffle=True)\nval_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = 16, shuffle=False)\n'

##Define Model

In [88]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from torchsummary import summary
ENCODER = 'efficientnet-b7'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid'
DEVICE = 'cuda'

model = smp.Unet(
    encoder_name = ENCODER,
    encoder_weights = ENCODER_WEIGHTS,
    in_channels = 3,
    classes = 1,
    activation = ACTIVATION,
)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth
100%|██████████| 254M/254M [00:01<00:00, 237MB/s]


RuntimeError: ignored

##Model Train

위쪽은 원래 train 코드와 동일합니다.


with torch.no_grad()--- 부분부터는 위에서 나눈 val 데이터 세트를 가져와 검증합니다


(이때는 학습을 하지 않습니다.)


loss: train에서 사용한 criterion

Accuracy는 픽셀별로 비교한 정확도 (정답픽셀/전체픽셀 *100)


In [89]:
best_loss = 1 # 매우 큰 값으로 초기값 가정
patience_limit = 5 # 몇 번의 epoch까지 지켜볼지를 결정
patience_check = 0 # 현재 몇 epoch 연속으로 loss 개선이 안되는지를 기록

In [ ]:
import segmentation_models_pytorch.utils
#%cd /content
# loss function과 optimizer 정의
criterion = smp.utils.losses.DiceLoss()
optimizer = torch.optim.Adam([dict(params=model.parameters(), lr=0.0001),])


#model.load_state_dict(torch.load('weights_only.pth'))
# training loop
result = []
correct=0
accuracy=0
loss1=0.35
for epoch in range(10):  # 10 에폭 동안 학습합니다.
    model.train()
    model.to('cuda')
    epoch_loss = 0
    for images, masks in tqdm(dataloader):
        images = images.float().to(device)
        masks = masks.float().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks.unsqueeze(1))
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {epoch_loss/len(dataloader)}')




    """ early stopping"""
    if loss1 > best_loss: # loss가 개선되지 않은 경우
        patience_check += 1

        if patience_check >= patience_limit: # early stopping 조건 만족 시 조기 종료
            break

    else: # loss가 개선된 경우
        best_loss = loss1
        patience_check = 0



    with torch.no_grad():
          model.eval()


          for images, masks in tqdm(val_dataloader):
              val_images = images.float().to(device)
              val_masks = masks.float().to(device) #torch.Size([8, 224, 224])

              outputs = model(val_images)
              new_masks = outputs.cpu().numpy() #numpy.ndarray 타입변경 (8, 1, 224, 224)
              new_masks = np.squeeze(new_masks, axis=1) #(8, 224, 224)
              new_masks = (new_masks > 0.35).astype(np.uint8) # Threshold = 0.35

          for i in range(len(val_images)):
              result.append(new_masks[i])

          result = torch.tensor(result)
          result = result.float().to(device)
          loss1 = criterion(result.unsqueeze(1), val_masks.unsqueeze(1))
          #확인용
          mul1=result*val_masks
          mul2=result+val_masks

          for i in range(len(val_images)):
            correct+= (mul1[i]==1).type(torch.float).sum().item()
            correct+= (mul2[i]==0).type(torch.float).sum().item()
            accuracy=100* (correct/(224*224*val_masks.shape[0]))

#모델 wight 저장
#%cd /content/drive/MyDrive/ai project 4-1
#torch.save(model.state_dict(), 'weights_only.pth')
#%cd /content
print(f'Epoch {epoch+1}, Loss: {epoch_loss/len(dataloader)}')
print(f'val_dice_score: {loss1},Accuracy: {accuracy}')


 35%|███▌      | 142/402 [07:29<14:48,  3.42s/it]

validation 예측 라벨과 정답 라벨을 출력할수 있습니다.

In [ ]:
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
figresult=result.to('cpu')
figmasks=masks.to('cpu')
for i in range(5):
  plt.figure(figsize=(10,10))
  plt.subplot(131)
  plt.imshow(figresult[i])
  plt.axis("off")
  plt.subplot(132)
  plt.imshow(figmasks[i])
  plt.axis("off")


##model save

In [ ]:
torch.save(model.state_dict(), 'weights_only.pth')

In [ ]:
model.load_state_dict(torch.load('weights_only.pth'))

##test model

In [ ]:
test_dataset = SatelliteDataset(csv_file='./test.csv', transform=transform_test, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)

In [ ]:
with torch.no_grad():
    model.eval()
    result = []
    for images in tqdm(test_dataloader):
        images = images.float().to(device)

        outputs = model(images)
        masks = outputs.cpu().numpy()
        masks = np.squeeze(masks, axis=1)
        masks = (masks > 0.35).astype(np.uint8) # Threshold = 0.35

        for i in range(len(images)):
            mask_rle = rle_encode(masks[i])
            if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                result.append(-1)
            else:
                result.append(mask_rle)

##Submission

csv파일 저장

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit['mask_rle'] = result

In [ ]:
#%cd /content/drive/MyDrive/ai project 4-1
submit.to_csv('./submit.csv', index=False)

결과물 출력

In [ ]:
#%cd /content
test_data = pd.read_csv('test.csv')
submit2 = pd.read_csv('submit (4).csv')
submit3 = pd.read_csv('try4_epoch10 & centercrop & efficientnet & dice loss.csv')
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

for i in range(550,580):
  test_image_path = test_data['img_path'][i]
  test_image = cv2.imread(test_image_path)
  test_mask2 = rle_decode(submit2['mask_rle'][i], (224,224))
  test_mask3 = rle_decode(submit3['mask_rle'][i], (224,224))

  plt.figure(figsize=(10,10))
  plt.subplot(131)
  plt.imshow(test_image)
  plt.axis("off")
  plt.subplot(132)
  plt.imshow(test_mask2)
  plt.axis("off")
  plt.subplot(133)
  plt.imshow(test_mask3)
  plt.axis("off")